In [1]:
import os, time, sys
from vllm import LLM, SamplingParams
sys.path.append('../elyra_pipeline/')
from postgres_utilities import connect_db, close_db, execute_sql_results, execute_sql_results_np

INFO 05-05 15:05:54 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model_name = "GritLM/GritLM-7B"
number_data_rows = 1000

In [3]:
batch_size = 100

In [4]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=50)
llm = LLM(model=model_name, max_model_len=4096)

INFO 05-05 15:06:05 [config.py:717] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 05-05 15:06:06 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-05 15:06:07 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='GritLM/GritLM-7B', speculative_config=None, tokenizer='GritLM/GritLM-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, c

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 05-05 15:06:13 [loader.py:458] Loading weights took 4.54 seconds
INFO 05-05 15:06:13 [gpu_model_runner.py:1347] Model loading took 13.4967 GiB and 4.967418 seconds
INFO 05-05 15:06:24 [backends.py:420] Using cache directory: /opt/app-root/src/.cache/vllm/torch_compile_cache/ce4dd6c153/rank_0_0 for vLLM's torch.compile
INFO 05-05 15:06:24 [backends.py:430] Dynamo bytecode transform time: 10.83 s
INFO 05-05 15:06:27 [backends.py:136] Cache the graph of shape None for later use
INFO 05-05 15:06:52 [backends.py:148] Compiling a graph for general shape takes 27.52 s
INFO 05-05 15:07:07 [monitor.py:33] torch.compile takes 38.35 s in total
INFO 05-05 15:07:07 [kv_cache_utils.py:634] GPU KV cache size: 38,832 tokens
INFO 05-05 15:07:07 [kv_cache_utils.py:637] Maximum concurrency for 4,096 tokens per request: 9.48x
INFO 05-05 15:07:36 [gpu_model_runner.py:1686] Graph capturing finished in 29 secs, took 0.51 GiB
INFO 05-05 15:07:36 [core.py:159] init engine (profile, create kv cache, warmup

In [5]:
try:
    conn = connect_db()
    sql = f"select productdisplayname from products limit {number_data_rows}"
    results = execute_sql_results_np(conn, sql)
except Exception as e:
    raise e
finally:
    close_db(conn)

In [6]:
test_description = "Starting batch generative test using local vllm"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Texts generated ...", end=" ")
messages = list()
for index, product in enumerate(results):
    description = product[0]
    prompt = f"Expand this product description: {description}"
    messages.append(prompt)

start_offset = 0
end_offset = batch_size
question_response_pairs = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    responses = llm.generate(messages[start_offset:end_offset], sampling_params)

    for r in responses:
        original_prompt = r.prompt
        generated_text = r.outputs[0].text
        assert len(generated_text) > 0
        question_response_pairs.append(f"{original_prompt}:{generated_text}")

    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

assert len(question_response_pairs) == number_data_rows

toc = time.perf_counter()
print("")
print(f"Time to generate text for {number_data_rows} rows: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch generative test using local vllm
-----------------------------------------------
Texts generated ... 0 100 

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

100 200 

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

200 300 

...

800 900 

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

900 1000 

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Time to generate text for 1000 rows: 29.4419 seconds
-----------------------------------------------
